# 2025 세법 개정본 반영

In [2]:
pip install jpype1

In [3]:
%pip install tabula-py

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
    --------------------------------------- 0.3/12.0 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.0 MB 4.5 MB/s eta 0:00:03
   ------- -------------------------------- 2.4/12.0 MB 4.6 MB/s eta 0:00:03
   ------------ --------------------------- 3.7/12.0 MB 5.1 MB/s eta 0:00:02
   ---------------- ----------------------- 5.0/12.0 MB 5.3 MB/s eta 0:00:02
   ------------------- -------------------- 5.8/12.0 MB 5.0 MB/s eta 0:00:02
   ----------------------- ---------------- 7.1/12.0 MB 5.2 MB/s eta 0:00:01
   -------------------------- ------------- 7.9/12.0 MB 5.1 MB/s eta 0:00:01
   ------------------------------ --------- 9.2/12.0 MB 5.2 MB/s eta 0:00:01
   ---------------------------------- ----- 10.5/12.0 MB 5.2 MB/s eta 0:00:01
   -------------------------------------- - 11.5/12.0 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 5.2 MB/s eta 0:00:00
Note: you 

In [17]:
pip install --upgrade langchain chromadb

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 2.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.22
    Uninstalling langchain-core-0.3.22:
      Successfully uninstalled langchain-core-0.3.22
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9
Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.schema import Document  # LangChain 문서 클래스
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from tabula import read_pdf
from dotenv import load_dotenv

load_dotenv()

# # PDF 파일 경로
# pdf_path = "data/tax_law/2024_new_tax.pdf"

# # SQLite 캐시 설정
# SQLiteCache(".cache_prompt.sqlite")

# # Tabula로 PDF에서 표 데이터 읽기
# tables = read_pdf(pdf_path, pages="all", lattice=True, multiple_tables=True)

# # 표 데이터를 텍스트로 변환
# table_texts = [table.to_string(index=False, header=True) for table in tables]

# # PyMuPDFLoader로 텍스트 데이터 로드
# text_loader = PyMuPDFLoader(pdf_path)
# texts = text_loader.load()  # 문서 리스트로 변환

# # 표 텍스트를 LangChain Document 형식으로 변환
# table_docs = [
#     Document(page_content=text, metadata={"source": f"Table {i + 1}"})
#     for i, text in enumerate(table_texts)
# ]

# # PyMuPDF에서 로드한 텍스트를 LangChain Document 형식으로 변환
# text_docs = [
#     Document(page_content=text.page_content, metadata=text.metadata)
#     for text in texts
# ]

# # 텍스트 문서와 표 문서를 결합
# all_docs = text_docs + table_docs
# # Split
# splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     model_name="gpt-4o-mini",
#     chunk_size=1000,
#     chunk_overlap=50,
# )

# # 문서 분리
# split_docs = splitter.split_documents(all_docs)

# # Vector Store 생성
# SQLiteCache(".cache_prompt.sqlite")  # 캐시 설정
# embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
# vector_store = Chroma.from_documents(
#     documents=split_docs,
#     embedding=embedding_model,
#     collection_name="tax",
#     persist_directory="vector_store/chroma/tax"
# )

# # Retriever 생성
# retriever = vector_store.as_retriever(
#     search_type="mmr",
#     search_kwargs={"k": 5, "fetch_k": 10}
# )


True

In [28]:
import os
from tqdm import tqdm

# SQLite 캐시 설정
SQLiteCache(".cache_prompt.sqlite")

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# PDF 파일 경로
pdf_dir = "data/tax_law" ## 실제 test용 
# pdf_dir = "data/tax_law_test"  # 돈아끼기 실험용

# 모든 PDF 파일 경로 가져오기
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

# 모든 문서 저장 리스트
all_split_docs = []

# PDF 파일 처리
for pdf_path in tqdm(pdf_files, desc="Processing PDF files"):
    try:
        # PyMuPDFLoader로 텍스트 데이터 로드
        text_loader = PyMuPDFLoader(pdf_path)
        texts = text_loader.load()  # 문서 리스트로 변환

        # 표 데이터를 Tabula로 읽기
        tables = read_pdf(pdf_path, pages="all", lattice=True, multiple_tables=True)
        table_texts = [table.to_string(index=False, header=True) for table in tables]

        # 표 데이터를 LangChain Document 형식으로 변환
        table_docs = [
            Document(page_content=text, metadata={"source": f"{pdf_path} - Table {i + 1}"})
            for i, text in enumerate(table_texts)
        ]

        # PyMuPDF에서 로드한 텍스트를 LangChain Document 형식으로 변환
        text_docs = [
            Document(page_content=text.page_content, metadata={"source": pdf_path})
            for text in texts
        ]

        # 텍스트 문서와 표 문서를 결합
        all_docs = text_docs + table_docs

        # Split
        splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            model_name="gpt-4o-mini",
            chunk_size=1000,
            chunk_overlap=50,
        )

        # 문서를 청크로 분리
        split_docs = splitter.split_documents(all_docs)
        all_split_docs.extend(split_docs)

    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")

# Vector Store 생성 및 저장
vector_store = Chroma.from_documents(
    documents=all_split_docs,
    embedding=embedding_model,
    collection_name="tax",
    persist_directory="vector_store/chroma/tax"  # 저장 경로
)

# 저장 작업은 Chroma 객체를 생성할 때 자동으로 처리됩니다.
print("모든 PDF File의의 작업이 완료되었습니다.")

Processing PDF files:   0%|          | 0/71 [00:00<?, ?it/s]c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 0 of document data/tax_law\2024_new_tax.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_law\2024_new_tax.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_law\2024_new_tax.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 17 of document data/tax_law\2024_new_tax.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchai

모든 PDF File의의 작업이 완료되었습니다.


In [29]:
split_docs[-3]

Document(metadata={'source': 'data/tax_law\\증권거래세법_시행령.pdf'}, page_content='법제처                                                            4                                                       국가법령정보센터\n증권거래세법 시행령\n4. 주권등 매매계약서 사본(법 제3조제3호의 규정에 의한 납세의무자에 한한다)\n5. 비과세양도명세서(법 제3조제1호의 규정에 의한 납세의무자에 한한다)\n6. 기타 참고사항\n②법 제10조제2항의 규정에 의한 증권거래세의 납부에 있어서는 증권거래세과세표준신고서와 함께 관할세무서장\n에게 납부하거나 「국세징수법」에 의한 납부서에 증권거래세과세표준신고서를 첨부하여 한국은행(그 대리점을 포\n함한다) 또는 체신관서에 납부하여야 한다.<개정 1996. 8. 22., 2005. 2. 19.>\n③납세의무자가 사업장을 폐지한 때에는 그날부터 25일 이내에 폐지한 날이 속하는 달의 과세표준과 세액을 제1항\n및 제2항의 규정에 의하여 신고하고 납부하여야 한다.<개정 2009. 2. 3.>\n④법 제3조제1호의 규정에 의한 납세의무자는 증권거래세 신고ㆍ납부기한 종료일의 다음날부터 6월 이내에 한하\n여 법 제10조제3항의 규정에 의한 환급을 할 수 있다.<신설 2000. 12. 29.>\n[제목개정 2000. 12. 29.]\n \n제8조(경정) ①법 제11조제2항 단서에서 “대통령령이 정하는 경우”라 함은 다음 각호의 1에 해당하는 경우를 말한다.\n<개정 2000. 12. 29.>\n1. 과세표준을 계산함에 있어서 필요한 장부 기타 증빙서류가 없거나 중요한 부분이 미비 또는 허위인 때\n2. 장부 기타의 증빙서류의 내용이 주권 등의 거래량에 비추어 신빙성이 없다고 인정될 때\n②법 제11조제2항 단서의 규정에 의하여 추계에 의하여 결정 또는 경정하는 때에는 조사에 의하여 확인된 주권 등

In [34]:
# Prompt Template 생성
messages = [
        ("ai", """
        You are an expert assistant on tax law.. Answer question using only the following context. 
        If you don't know the answer, just say you don't know. Don't make it up. 
        Answer in Korean.
        
        {context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o-mini")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser



In [37]:
chain.invoke("친환경자동차의 개별소비세에 대해 변한 사항이 있어? ")

'네, 친환경자동차의 개별소비세에 대한 변동 사항이 있습니다. \n\n1. **감면 적용기한 연장 및 재설계**: \n   - 하이브리드차, 전기차, 수소전기차에 대해 개별소비세 감면이 적용됩니다.\n   - 하이브리드차는 대당 100만원에서 70만원으로 감면 한도가 축소되었습니다.\n   - 적용기한은 2024년 12월 31일까지입니다.\n\n2. **감면 한도**:\n   - 하이브리드차: 대당 70만원\n   - 전기차: 대당 300만원\n   - 수소전기차: 대당 400만원\n\n이러한 변경 사항은 2025년 1월 1일 이후 반출하거나 수입신고하는 경우부터 적용됩니다.'

In [36]:
chain.invoke("의료기술협력단을 기부금 손금산입 대상에 대해 변한 사항이 있어? ")


'의료기술협력단을 기부금 손금산입 대상에 포함하는 사항이 변했습니다. 구체적으로, 특례기부금 단체 및 일반기부금 단체의 범위에 의료기술협력단이 추가되었습니다. \n\n- 특례기부금 단체에는 국가, 지방자치단체, 국립대학병원, 서울대학교병원, 사립학교가 운영하는 병원 등이 포함되며, 여기서 의료기술협력단이 설립한 병원이 추가되었습니다.\n- 일반기부금 단체에도 사회복지법인, 어린이집, 유치원, 의료법인이 포함되며, 일반기부금 단체에 해당하는 병원이 설립하는 의료기술협력단도 추가되었습니다.\n\n적용 시기는 특례기부금은 2025년 1월 1일 이후 신고하는 분부터 적용되며, 일반기부금은 영 시행일 이후 신고하는 분부터 적용됩니다.'

In [32]:
import json

with open("processed_data.json", "w", encoding="utf-8") as f:
    json.dump([doc.dict() for doc in all_split_docs], f, ensure_ascii=False, indent=4)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_6964\769843488.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json.dump([doc.dict() for doc in all_split_docs], f, ensure_ascii=False, indent=4)
